In [1]:
#data
import Pkg
Pkg.add("JuMP")
using JuMP
import Pkg; Pkg.add("GLPK") 
using GLPK 
using LinearAlgebra
using DelimitedFiles
Pkg.add("FilePaths")  
#constants
phi = 1000000 #budget
relax = [] #vector of relaxation problems
k = 1 #overall iteration counter
HORIZON = 3 #time HORIZON 
T = HORIZON #same thing as horizon but more consise notation
GENERATORS = 6 #number of generators
QS = 6 #number of quick start generators
BUSES = 6 #number of BUSES
ARCS= 7
BREAKPOINTS = 5 
#reference bus is bus no. 5 
#for testing:
relax_obj = []
sub_obj = []
#READ IN DATA
SF_data=readdlm("/Users/WilliamYang/Desktop/research/data/6bus_data/6bus_SF_b_l.txt") #'distribution' data for shif factor from MATLAB output
bus_data=readdlm("/Users/WilliamYang/Desktop/research/data/6bus_data/6bus_data.txt") #bus data from Excel sheet
generator_data =readdlm("/Users/WilliamYang/Desktop/research/data/6bus_data/6bus_generator_data.txt") #generator data from Excel sheet
qs_data =readdlm("/Users/WilliamYang/Desktop/research/data/6bus_data/6bus_quickstart_data.txt") #qucik start data adapted from generator data
inertia_data=readdlm("/Users/WilliamYang/Desktop/research/data/6bus_data/6bus_inertia_data.txt") #inertia data from Excel sheet
alpha_data = readdlm("/Users/WilliamYang/Desktop/research/data/6bus_data/6bus_alpha.txt") #alpha and beta data from MATLAB output
beta_data = readdlm("/Users/WilliamYang/Desktop/research/data/6bus_data/6bus_beta.txt")
transmission_data=readdlm("/Users/WilliamYang/Desktop/research/data/6bus_data/6bus_arc_data.txt") #arc data from Excel sheet
demand_data=readdlm("/Users/WilliamYang/Desktop/research/data/6bus_data/6bus_demand_data.txt")  #demand data
req_data=readdlm("/Users/WilliamYang/Desktop/research/data/6bus_data/6bus_req_data.txt")  #demand data

#BUS DATA:
FFR = bus_data[2,:]
deleteat!(FFR,1)
M_ffr = sum(FFR) #big M


println("read in raw data")
#GENERATOR DATA:
generator_data = generator_data[1:end, 1:end.!=1 ] #remove first column of labels
qs_data = qs_data[1:end, 1:end.!=1 ] #remove first column of labels

#initialize each data array
G = Array{Any}(undef,BUSES) #this is a set of all generators in each bus (for reference)
G_prime = Array{Any}(undef,BUSES) #this is a set of all quick start generators in each bus (for reference)


#read in generator data
Pmin = generator_data[2,:]
Pmax=generator_data[3,:]
REGU=generator_data[4,:]
REGD=generator_data[5,:]
RMT=generator_data[6,:]
SU=generator_data[7,:]
SD=generator_data[8,:]
Tminu=generator_data[9,:]
Tmind=generator_data[10,:]
RU=generator_data[11,:]
RD=generator_data[12,:]
RU_bar=generator_data[13,:]
RD_bar=generator_data[14,:]
NSP=generator_data[15,:]
H=generator_data[16,:]
S=generator_data[17,:]

Pmin_prime=qs_data[2,:] 
Pmax_prime=qs_data[3,:]
REGU_prime=qs_data[4,:]
REGD_prime=qs_data[5,:]
RMT_prime=qs_data[6,:]
SU_prime=qs_data[7,:]
SD_prime=qs_data[8,:]
Tminu_prime=qs_data[9,:]
Tmind_prime=qs_data[10,:]
RU_prime=qs_data[11,:]
RD_prime=qs_data[12,:]
RU_bar_prime=qs_data[13,:]
RD_bar_prime=qs_data[14,:]
NSP_prime=qs_data[15,:]
H_prime=qs_data[16,:]
S_prime=qs_data[17,:]


println("imported generator data")

req_data = req_data[1:end, 1:end.!=1 ] #remove first column of labels
RREGU=req_data[2,:]
RREGD=req_data[3,:]
RNSP=req_data[4,:]

#define G
for b = 1:BUSES
    G[b]=[]
    G_prime[b]=[]
end

for n =1:GENERATORS
    b = generator_data[1,n] #this is the bus that generator n belongs to (read in from data)
    push!(G[b],n)
end

for m=1:QS
    b = qs_data[1,m]
    push!(G_prime[b],m)
end



#INERTIA DATA:
K = inertia_data[:,7] #K_sg aka 'equivalency ratio'
deleteat!(K,1)
IN = inertia_data[:,2]
deleteat!(IN,1)

RFRR_1 = inertia_data[2,3]

IR = inertia_data[:,8]
deleteat!(IR,1)
println("imported inertia data")


#SHIF FACTOR DATA:
SF = zeros(Float32,BUSES,ARCS)
counter = 1
for b =1:BUSES
    for l = 1:ARCS
        SF[b,l] = SF_data[counter]
        counter = counter +1
    end
end
println("imported shif factor data")


#ALPHA and BETA VALUES:
alpha = zeros(Float32,GENERATORS,BREAKPOINTS)
beta = zeros(Float32,GENERATORS,BREAKPOINTS)
alpha_prime = zeros(Float32,QS,BREAKPOINTS)
beta_prime = zeros(Float32,QS,BREAKPOINTS)
counter = 1
for n = 1:GENERATORS
    for j = 1:BREAKPOINTS
        alpha[n,j]=alpha_data[counter]
        beta[n,j]=beta_data[counter]
        
        counter = counter+1
    end
end
counter = 1
for m = 1:QS
    for j = 1:BREAKPOINTS
        alpha_prime[m,j]=alpha_data[counter]
        beta_prime[m,j]=beta_data[counter]
        
        counter = counter+1
    end
end


println("imported alpha and beta values for piecewise approximation (for this example, we assume quick start and conventional generators have the same fuel costs)")

#TRANSMISSION LINE DATA:
new_index_1 = transmission_data[:,2]
deleteat!(new_index_1,1)
new_index_2 = transmission_data[:,3]
deleteat!(new_index_2,1)
reactance = transmission_data[:,4]
deleteat!(reactance,1)
F = transmission_data[:,5]
deleteat!(F,1)



println("imported transmission line data")

#demand data
demand_data = demand_data[1:end, 1:end.!=1 ]
demand_data = demand_data[1:end.!=1, 1:end ]#remove first row and column of labels
d_hat=transpose(demand_data)


println("finished importing data")


#more constants
M = 100 #bigM
INTERVALS = length(IN)

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


┌ Warning: Some registries failed to update:
│     — /Users/WilliamYang/.julia/registries/General — failed to fetch from repo
└ @ Pkg.Types /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.4/Pkg/src/Types.jl:1122
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


read in raw data
imported generator data
imported inertia data
imported shif factor data
imported alpha and beta values for piecewise approximation (for this example, we assume quick start and conventional generators have the same fuel costs)
imported transmission line data
finished importing data


13

In [2]:
#MASTER PROBLEM
master =Model(GLPK.Optimizer)
@variable(master,gamma_plus>=0) #uncertainty set size
@variable(master,gamma_minus[1:BUSES,1:HORIZON]>=0) #uncertainty set size

x=Array{Any}(undef,GENERATORS,HORIZON)#variable containers
y=Array{Any}(undef,GENERATORS,HORIZON)
z=Array{Any}(undef,GENERATORS,HORIZON)

for t = 1:HORIZON
    for n = 1:GENERATORS
        x[n,t] = @variable(master,base_name=string("x_",string(n),"_",string(t)),binary = true)
        y[n,t] = @variable(master,base_name=string("y_",string(n),"_",string(t)),binary = true)
        z[n,t]= @variable(master,base_name=string("z_",string(n),"_",string(t)),binary = true)
    end
end

@objective(master,Max,gamma_plus+sum(gamma_minus[b,t] for b=1:BUSES, t=1:HORIZON))
for t = 1:HORIZON
    for n = 1:GENERATORS            
        @constraint(master,x[n,t] + z[n,t]<=1)#can't turn on and off            
        if t >=2   
        @constraint(master,y[n,t]-y[n,t-1]==x[n,t]-z[n,t]) #logically define relationship between x,y, and z 
        end
        if t >= Tminu[n]
            @constraint(master,sum(x[n,tau] for tau = t-Tminu[n]+1:t) <= y[n,t]) #min up constraint
        end
        if t >= Tmind[n]
            @constraint(master,sum(z[n,tau] for tau = t-Tmind[n]+1:t) <= 1-y[n,t]) #min down constraint
        end
    end
end
#boundary case
for n = 1:GENERATORS
    @constraint(master,y[n,1]==x[n,1]-z[n,1])
end
@constraint(master,gamma_plus<=M) #to ensure the problem is bounded
for b=1:BUSES
    for t=1:HORIZON
        @constraint(master,gamma_minus[b,t]<=d_hat[b,t])
    end
end



println("======================================MASTER PROBLEM===============================================")
println(master)
println("-----------------------------------solution to master problem:-----------------------------------")


optimize!(master)
#store x_hat, y_hat, z_hat, gamma_hat
x_hat =  Array{Any}(undef,GENERATORS,HORIZON)
y_hat = Array{Any}(undef,GENERATORS,HORIZON)
z_hat = Array{Any}(undef,GENERATORS,HORIZON)

gamma_plus_hat = JuMP.value.(gamma_plus)
gamma_minus_hat = JuMP.value.(gamma_minus)
for t = 1:HORIZON
    for n = 1:GENERATORS
        x_hat[n,t]=JuMP.value(x[n,t])
        y_hat[n,t]=JuMP.value(y[n,t])
        z_hat[n,t]=JuMP.value(z[n,t])
    end
end


println("gamma_hat+: ",gamma_plus_hat)
println("gamma_hat-: ",gamma_minus_hat)
println("x_hat: ",x_hat)
println("y_hat: ",y_hat)
println("z_hat: ",z_hat)
println("---------------------------------------------------------------------------------------------------")



======================================MASTER PROBLEM===============================================
Max gamma_plus + gamma_minus[1,1] + gamma_minus[1,2] + gamma_minus[1,3] + gamma_minus[2,1] + gamma_minus[2,2] + gamma_minus[2,3] + gamma_minus[3,1] + gamma_minus[3,2] + gamma_minus[3,3] + gamma_minus[4,1] + gamma_minus[4,2] + gamma_minus[4,3] + gamma_minus[5,1] + gamma_minus[5,2] + gamma_minus[5,3] + gamma_minus[6,1] + gamma_minus[6,2] + gamma_minus[6,3]
Subject to
 -y_1_1 - x_1_2 + y_1_2 + z_1_2 = 0.0
 -y_2_1 - x_2_2 + y_2_2 + z_2_2 = 0.0
 -y_3_1 - x_3_2 + y_3_2 + z_3_2 = 0.0
 -y_4_1 - x_4_2 + y_4_2 + z_4_2 = 0.0
 -y_5_1 - x_5_2 + y_5_2 + z_5_2 = 0.0
 -y_6_1 - x_6_2 + y_6_2 + z_6_2 = 0.0
 -y_1_2 - x_1_3 + y_1_3 + z_1_3 = 0.0
 -y_2_2 - x_2_3 + y_2_3 + z_2_3 = 0.0
 -y_3_2 - x_3_3 + y_3_3 + z_3_3 = 0.0
 -y_4_2 - x_4_3 + y_4_3 + z_4_3 = 0.0
 -y_5_2 - x_5_3 + y_5_3 + z_5_3 = 0.0
 -y_6_2 - x_6_3 + y_6_3 + z_6_3 = 0.0
 -x_1_1 + y_1_1 + z_1_1 = 0.0
 -x_2_1 + y_2_1 + z_2_1 = 0.0
 -x_3_1 + y_3_1 

In [3]:
#fix 2nd stage binary variables to begin CCGA q1


x_prime = [1.0 0.0 0.0; 0.0 0.0 0.0; 1.0 0.0 0.0; 1.0 0.0 0.0; 1.0 0.0 0.0; 1.0 0.0 0.0]
y_prime=[1.0 1.0 1.0; 0.0 0.0 0.0; 1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0]
z_prime =[0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
delta = [1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
demand_hat = [300.0 350.0 200.0; 225.0 250.0 200.0; 200.0 250.0 225.0; 175.0 200.0 175.0; 150.0 200.0 175.0000000000004; 125.0 200.0 150.0]

6×3 Array{Float64,2}:
 300.0  350.0  200.0
 225.0  250.0  200.0
 200.0  250.0  225.0
 175.0  200.0  175.0
 150.0  200.0  175.0
 125.0  200.0  150.0

Any[700, 400, 300, 700, 700, 300]


In [5]:
#CREATE RELAXATION PROBLEM
@eval $(Symbol(:m,k)) =Model(GLPK.Optimizer)

#Model(GLPK.Optimizer)
#Model(optimizer_with_attributes(GLPK.Optimizer, "msg_lev" => GLPK.MSG_ALL))
temp = @eval $(Symbol(:m, k))
push!(relax, temp) #add a list of problems
@variable(relax[1],eta) #auxilary variable for CCG
@variable(relax[1],demand[1:BUSES,1:HORIZON]) #worst-case demand variable

r = 1#we'll use this as an iteration count for each variable (same as in the formulation doc)

1

In [6]:
#------------------------variables------------------------
#initialize empty variable arrays to be filled throughout CCGA

#primal variables 

c = []
c_prime = []

#generation variables
p =  []
regu = []
regd=[]
sr= []
nsp =[]
#quick start generation variables
p_prime = []
regu_prime = []
regd_prime = []
sr_prime=[]
nsp_prime = []

p_bus = [] #auxilary variable to denote the total electricity for each bus


#frequency response variables
ffr = []
tffr = []
frr = []
rfrr =[]

inx = []

#slack variables

s_budget_minus = []
s_balance_plus = []
s_balance_minus = []
s_Pmin_plus=[]
s_prime_Pmin_plus = []
s_transUB_minus= []
s_transLB_plus = []
s_RREGU_plus =  []
s_RREGD_plus =  []
s_RNSP_plus =  []
s_rfrr_plus =  []
s_tffr1_plus =  []
s_tffr2_minus = []
s_inx1_plus =[]
s_inx2_minus = []
#dual variables
lambda_budget =  []
lambda_fuel= []
lambda_prime_fuel= []
lambda_RU = []
lambda_prime_RU = []
lambda_RD = []
lambda_prime_RD= []
lambda_sr= []
lambda_prime_sr=[]
lambda_Pmax= []
lambda_prime_Pmax= []
lambda_Pmin= []
lambda_prime_Pmin= []
lambda_nsp = []
lambda_prime_nsp= []
lambda_regu= []
lambda_prime_regu=[]
lambda_regd= []
lambda_prime_regd= []
lambda_ffr= []
lambda_balance = []
lambda_transUB= []
lambda_transLB = []
lambda_RREGUreq = []
lambda_RREGDreq = []
lambda_NSPreq = []
lambda_rfrrreq = []
lambda_frr = []
lambda_tffr1 = []
lambda_tffr2= []
lambda_rfrr =[]
lambda_inx1 = []
lambda_inx2= []
lambda_inertia =[]



#We split up the dual objective into many auxilary variable
dual_obj_budget=[]
dual_obj_fuel=[]
dual_obj_capacity1 = []
dual_obj_capacity1_prime = []
dual_obj_capacity2 = []
dual_obj_capacity2_prime = []
dual_obj_ffr = []
dual_obj_trans = []
dual_obj_req = []
dual_obj_tffr = []
dual_obj_rfrr = []
dual_obj_inertia1 = []
dual_obj_inertia2 = []
slack =[]

0-element Array{Any,1}

In [7]:
#------------------------variables------------------------
#initialize variables for each iteration
#primal variables 
#this set of variables are arrays of vectors
c_temp = Array{Any}(undef,GENERATORS,HORIZON) #cost 
c_prime_temp = Array{Any}(undef,QS,HORIZON)
#generation variables
p_temp =  Array{Any}(undef,GENERATORS,HORIZON)
regu_temp =Array{Any}(undef,GENERATORS,HORIZON)
regd_temp =Array{Any}(undef,GENERATORS,HORIZON)
sr_temp= Array{Any}(undef,GENERATORS,HORIZON)
nsp_temp = Array{Any}(undef,GENERATORS,HORIZON)
#quick start generation variables
p_prime_temp = Array{Any}(undef,QS,HORIZON)
regu_prime_temp = Array{Any}(undef,QS,HORIZON)
regd_prime_temp = Array{Any}(undef,QS,HORIZON)
sr_prime_temp =Array{Any}(undef,QS,HORIZON)
nsp_prime_temp = Array{Any}(undef,QS,HORIZON)

p_bus_temp = Array{Any}(undef,BUSES,HORIZON)

#frequency response variables
ffr_temp = Array{Any}(undef,BUSES,HORIZON)
tffr_temp = Array{Any}(undef,HORIZON)
frr_temp = Array{Any}(undef,HORIZON)
rfrr_temp = Array{Any}(undef,HORIZON)
inx_temp = Array{Any}(undef,INTERVALS,HORIZON)


#slack variables
s_budget_minus_temp =  @variable(relax[1],base_name=string("s_budget_minus^",string(r))) #slack variable
s_balance_plus_temp = Array{Any}(undef,HORIZON)
s_balance_minus_temp = Array{Any}(undef,HORIZON)
s_Pmin_plus_temp = Array{Any}(undef,GENERATORS,HORIZON)
s_prime_Pmin_plus_temp = Array{Any}(undef,QS,HORIZON)
s_transUB_minus_temp= Array{Any}(undef,ARCS,HORIZON)
s_transLB_plus_temp = Array{Any}(undef,ARCS,HORIZON)
s_RREGU_plus_temp =  Array{Any}(undef,HORIZON)
s_RREGD_plus_temp =  Array{Any}(undef,HORIZON)
s_RNSP_plus_temp =  Array{Any}(undef,HORIZON)
s_rfrr_plus_temp =  Array{Any}(undef,HORIZON)
s_tffr1_plus_temp =  Array{Any}(undef,INTERVALS-1,HORIZON)
s_tffr2_minus_temp = Array{Any}(undef,INTERVALS-1,HORIZON)
s_inx1_plus_temp = Array{Any}(undef,INTERVALS-1,HORIZON)
s_inx2_minus_temp = Array{Any}(undef,INTERVALS-1,HORIZON)

#dual variables

lambda_budget_temp =  @variable(relax[1],base_name=string("lambda_budget^",string(r))) 
lambda_fuel_temp= Array{Any}(undef,BREAKPOINTS,GENERATORS,HORIZON)
lambda_prime_fuel_temp= Array{Any}(undef,BREAKPOINTS,QS,HORIZON)

lambda_RU_temp = Array{Any}(undef,GENERATORS,HORIZON)
lambda_prime_RU_temp = Array{Any}(undef,QS,HORIZON)
lambda_RD_temp = Array{Any}(undef,GENERATORS,HORIZON)
lambda_prime_RD_temp= Array{Any}(undef,QS,HORIZON)
lambda_sr_temp= Array{Any}(undef,GENERATORS,HORIZON)
lambda_prime_sr_temp= Array{Any}(undef,QS,HORIZON)
lambda_Pmax_temp= Array{Any}(undef,GENERATORS,HORIZON)
lambda_prime_Pmax_temp= Array{Any}(undef,QS,HORIZON)
lambda_Pmin_temp= Array{Any}(undef,GENERATORS,HORIZON)
lambda_prime_Pmin_temp= Array{Any}(undef,QS,HORIZON)
lambda_nsp_temp = Array{Any}(undef,GENERATORS,HORIZON)
lambda_prime_nsp_temp= Array{Any}(undef,QS,HORIZON)
lambda_regu_temp= Array{Any}(undef,GENERATORS,HORIZON)
lambda_prime_regu_temp= Array{Any}(undef,QS,HORIZON)
lambda_regd_temp= Array{Any}(undef,GENERATORS,HORIZON)
lambda_prime_regd_temp= Array{Any}(undef,QS,HORIZON)

lambda_ffr_temp= Array{Any}(undef,BUSES,HORIZON)
lambda_balance_temp = Array{Any}(undef,HORIZON)
lambda_transUB_temp= Array{Any}(undef,ARCS,HORIZON)
lambda_transLB_temp = Array{Any}(undef,ARCS,HORIZON)
lambda_RREGUreq_temp = Array{Any}(undef,HORIZON)
lambda_RREGDreq_temp = Array{Any}(undef,HORIZON)
lambda_NSPreq_temp = Array{Any}(undef,HORIZON)
lambda_rfrrreq_temp = Array{Any}(undef,HORIZON)
lambda_frr_temp = Array{Any}(undef,HORIZON)
lambda_tffr1_temp = Array{Any}(undef,INTERVALS-1,HORIZON)
lambda_tffr2_temp= Array{Any}(undef,INTERVALS-1,HORIZON)
lambda_rfrr_temp = Array{Any}(undef,HORIZON)
lambda_inx1_temp = Array{Any}(undef,INTERVALS-1,HORIZON)
lambda_inx2_temp= Array{Any}(undef,INTERVALS-1,HORIZON)
lambda_inertia_temp =  Array{Any}(undef,HORIZON)


#----------------create variable instances---------------------------
#primal

for t = 1: HORIZON
    for n= 1: GENERATORS
        c_temp[n,t] =@variable(relax[1],base_name=string("c_",string(n),"_",string(t),"^",string(r)))   
        p_temp[n,t] =@variable(relax[1],base_name=string("p_",string(n),"_",string(t),"^",string(r)))                    
        regu_temp[n,t] =@variable(relax[1],base_name=string("regu_",string(n),"_",string(t),"^",string(r))) 
        regd_temp[n,t] =@variable(relax[1],base_name=string("regd_",string(n),"_",string(t),"^",string(r))) 
        sr_temp[n,t] =@variable(relax[1],base_name=string("sr_",string(n),"_",string(t),"^",string(r)))   
        nsp_temp[n,t] =@variable(relax[1],base_name=string("nsp_",string(n),"_",string(t),"^",string(r)))   
    end
end


for t = 1:HORIZON
    for m= 1: BUSES
        c_prime_temp[m,t] =@variable(relax[1],base_name=string("c_prime_",string(m),"_",string(t),"^",string(r)))   
        p_prime_temp[m,t] =@variable(relax[1],base_name=string("p_prime_",string(m),"_",string(t),"^",string(r)))   
        regu_prime_temp[m,t] =@variable(relax[1],base_name=string("regu_prime_",string(m),"_",string(t),"^",string(r)))     
        regd_prime_temp[m,t] =@variable(relax[1],base_name=string("regd_prime_",string(m),"_",string(t),"^",string(r)))     
        sr_prime_temp[m,t] =@variable(relax[1],base_name=string("sr_prime_",string(m),"_",string(t),"^",string(r)))               
        nsp_prime_temp[m,t] =@variable(relax[1],base_name=string("nsp_prime_",string(m),"_",string(t),"^",string(r)))   
    end       
end




for b = 1: BUSES
    for t=1:HORIZON
        ffr_temp[b,t] =@variable(relax[1],base_name=string("ffr_prime_",string(b),"_",string(t),"^",string(r)))  
        p_bus_temp[b,t]=@variable(relax[1],base_name=string("p_bus_",string(b),"_",string(t),"^",string(r)))  
    end
end

push!(ffr,ffr_temp)
push!(p_bus,p_bus_temp)
for t = 1:HORIZON
    tffr_temp[t] =@variable(relax[1],base_name=string("tffr_",string(t),"^",string(r)))   
    frr_temp[t] =@variable(relax[1],base_name=string("frr_",string(t),"^",string(r)))

    rfrr_temp[t] =@variable(relax[1],base_name=string("rfrr_",string(t),"^",string(r)))



    for sg=1:INTERVALS
        inx_temp[sg,t] =@variable(relax[1],base_name=string("inx_",string(sg),"_",string(t),"^",string(r)))
    end
end
#cost 
push!(c,c_temp )  
#push!(jeff,jeff_temp)
push!( c_prime,c_prime_temp )   


#generation variables
push!(p,p_temp)  
push!(regu,regu_temp)  
push!(regd,regd_temp)  
push!(sr,sr_temp)   
push!(nsp,nsp_temp )  

#quick start generation variables
push!(p_prime,p_prime_temp )   
push!(regu_prime,regu_prime_temp )   
push!(regd_prime,regd_prime_temp )

push!(sr_prime,sr_prime_temp )  
push!(nsp_prime,nsp_prime_temp )  
push!(p_bus,p_bus_temp)  



#frequency response variables

push!(tffr,tffr_temp)
push!(frr,frr_temp)
push!(rfrr,rfrr_temp)

push!(inx,inx_temp)


#slack

for t =1:HORIZON
    s_balance_plus_temp[t] = @variable(relax[1],base_name=string("s_balance_plus_",string(t),"^",string(r)))
    s_balance_minus_temp[t]  = @variable(relax[1],base_name=string("s_balance_minus_",string(t),"^",string(r)))
    s_RREGU_plus_temp[t]  = @variable(relax[1],base_name=string("s_RREGU_plus_",string(t),"^",string(r)))
    s_RREGD_plus_temp[t]  = @variable(relax[1],base_name=string("s_RREGD_plus_",string(t),"^",string(r)))
    s_RNSP_plus_temp[t]  = @variable(relax[1],base_name=string("s_RNSP_plus_",string(t),"^",string(r)))
    s_rfrr_plus_temp[t]  = @variable(relax[1],base_name=string("s_rfrr_plus_",string(t),"^",string(r)))
    for n =1:GENERATORS
        s_Pmin_plus_temp[n,t]=@variable(relax[1],base_name=string("s_Pmin_plus_",string(n),"_",string(t),"^",string(r)))
    end
    for m =1:QS
        s_prime_Pmin_plus_temp[m,t]=@variable(relax[1],base_name=string("s_prime_Pmin_plus_",string(m),"_",string(t),"^",string(r)))
    end
    for l=1:ARCS
        s_transUB_minus_temp[l,t]  = @variable(relax[1],base_name=string("s_transUB_minus_",string(l),"_",string(t),"^",string(r)))
        s_transLB_plus_temp[l,t] = @variable(relax[1],base_name=string("s_transLB_plus_",string(l),"_",string(t),"^",string(r)))
    end
    for sg = 1:INTERVALS-1
        s_tffr1_plus_temp[sg,t] = @variable(relax[1],base_name=string("s_tffr1_plus_",string(sg),"_",string(t),"^",string(r)))
        s_tffr2_minus_temp[sg,t] = @variable(relax[1],base_name=string("s_tffr2_minus_",string(sg),"_",string(t),"^",string(r)))
        s_inx1_plus_temp[sg,t] = @variable(relax[1],base_name=string("s_inx1_plus_",string(sg),"_",string(t),"^",string(r)))
        s_inx2_minus_temp[sg,t] = @variable(relax[1],base_name=string("s_inx2_minus_",string(sg),"_",string(t),"^",string(r)))
    end  
end


push!(s_budget_minus,s_budget_minus_temp)
push!(s_balance_plus,s_balance_plus_temp)
push!(s_balance_minus,s_balance_minus_temp)
push!(s_Pmin_plus,s_Pmin_plus_temp)
push!(s_prime_Pmin_plus,s_prime_Pmin_plus_temp)
push!(s_RREGU_plus,s_RREGU_plus_temp)
push!(s_RREGD_plus,s_RREGD_plus_temp)
push!(s_RNSP_plus,s_RNSP_plus_temp)
push!(s_rfrr_plus,s_rfrr_plus_temp)
push!(s_transUB_minus,s_transUB_minus_temp)
push!(s_transLB_plus,s_transLB_plus_temp)
push!(s_tffr1_plus,s_tffr1_plus_temp)
push!(s_tffr2_minus,s_tffr2_minus_temp)
push!(s_inx1_plus,s_inx1_plus_temp)
push!(s_inx2_minus,s_inx2_minus_temp)

#dual

for t =1:HORIZON
    lambda_RREGUreq_temp[t] = @variable(relax[1],base_name = string("lambda_RREGUreq_",string(t),"^",string(r)))
    lambda_RREGDreq_temp[t] = @variable(relax[1],base_name = string("lambda_RREGDreq_",string(t),"^",string(r)))
    lambda_NSPreq_temp[t]=@variable(relax[1],base_name = string("lambda_NSPreq_",string(t),"^",string(r)))
    lambda_rfrrreq_temp[t]=@variable(relax[1],base_name = string("lambda_rfrrreq_",string(t),"^",string(r)))
    lambda_balance_temp[t]=@variable(relax[1],base_name = string("lambda_balance_",string(t),"^",string(r)))
    lambda_frr_temp[t]=@variable(relax[1],base_name = string("lambda_frr_",string(t),"^",string(r)))
    lambda_rfrr_temp[t]=@variable(relax[1],base_name = string("lambda_rfrr_",string(t),"^",string(r)))
    lambda_inertia_temp[t]=@variable(relax[1],base_name = string("lambda_inertia_",string(t),"^",string(r)))
    for n=1:GENERATORS
        lambda_RU_temp[n,t]=@variable(relax[1],base_name = string("lambda_RU_",string(n),"_",string(t),"^",string(r)))
        lambda_RD_temp[n,t]=@variable(relax[1],base_name = string("lambda_RD_",string(n),"_",string(t),"^",string(r)))
        lambda_sr_temp[n,t]=@variable(relax[1],base_name = string("lambda_sr_",string(n),"_",string(t),"^",string(r)))

        lambda_Pmax_temp[n,t]=@variable(relax[1],base_name = string("lambda_Pmax_",string(n),"_",string(t),"^",string(r)))
        lambda_Pmin_temp[n,t]=@variable(relax[1],base_name = string("lambda_Pmin_",string(n),"_",string(t),"^",string(r)))
        lambda_nsp_temp[n,t]=@variable(relax[1],base_name = string("lambda_nsp_",string(n),"_",string(t),"^",string(r)))
        lambda_regu_temp[n,t]=@variable(relax[1],base_name = string("lambda_regu_",string(n),"_",string(t),"^",string(r)))
        lambda_regd_temp[n,t]=@variable(relax[1],base_name = string("lambda_regd_",string(n),"_",string(t),"^",string(r)))
        for j = 1:BREAKPOINTS
            lambda_fuel_temp[j,n,t]=@variable(relax[1],base_name = string("lambda_fuel_",string(j),"_",string(n),"_",string(t),"^",string(r)))
        end
    end


    for m=1:QS
        lambda_prime_RU_temp[m,t]=@variable(relax[1],base_name = string("lambda_prime_RU_",string(m),"_",string(t),"^",string(r)))
        lambda_prime_RD_temp[m,t]=@variable(relax[1],base_name = string("lambda_prime_RD_",string(m),"_",string(t),"^",string(r)))
        lambda_prime_sr_temp[m,t]=@variable(relax[1],base_name = string("lambda_prime_sr_",string(m),"_",string(t),"^",string(r)))
        lambda_prime_Pmax_temp[m,t]=@variable(relax[1],base_name = string("lambda_prime_Pmax_",string(m),"_",string(t),"^",string(r)))
        lambda_prime_Pmin_temp[m,t]=@variable(relax[1],base_name = string("lambda_prime_Pmin_",string(m),"_",string(t),"^",string(r)))
        lambda_prime_nsp_temp[m,t]=@variable(relax[1],base_name = string("lambda_prime_nsp_",string(m),"_",string(t),"^",string(r)))
        lambda_prime_regu_temp[m,t]=@variable(relax[1],base_name = string("lambda_prime_regu_",string(m),"_",string(t),"^",string(r)))
        lambda_prime_regd_temp[m,t]=@variable(relax[1],base_name = string("lambda_prime_regd_",string(m),"_",string(t),"^",string(r)))
        for j=1:BREAKPOINTS
            lambda_prime_fuel_temp[j,m,t]=@variable(relax[1],base_name = string("lambda_prime_fuel_",string(j),"_",string(m),"_",string(t),"^",string(r)))
        end
    end

    for b = 1:BUSES
        lambda_ffr_temp[b,t]=@variable(relax[1],base_name = string("lambda_ffr_",string(b),"_",string(t),"^",string(r)))
    end

    for l = 1:ARCS
        lambda_transUB_temp[l,t]=@variable(relax[1],base_name = string("lambda_transUB_",string(l),"_",string(t),"^",string(r)))
        lambda_transLB_temp[l,t]=@variable(relax[1],base_name = string("lambda_transLB_",string(l),"_",string(t),"^",string(r)))
    end
    for sg=1:INTERVALS-1
        lambda_tffr1_temp[sg,t]=@variable(relax[1],base_name = string("lambda_tffr1_",string(sg),"_",string(t),"^",string(r)))     
        lambda_inx1_temp[sg,t]=@variable(relax[1],base_name = string("lambda_inx1_",string(sg),"_",string(t),"^",string(r)))
        lambda_tffr2_temp[sg,t]=@variable(relax[1],base_name = string("lambda_tffr2_",string(sg),"_",string(t),"^",string(r)))
        lambda_inx2_temp[sg,t]=@variable(relax[1],base_name = string("lambda_inx2_",string(sg),"_",string(t),"^",string(r)))
    end
end


push!(lambda_budget,lambda_budget_temp)
push!(lambda_fuel,lambda_fuel_temp)
push!(lambda_prime_fuel,lambda_prime_fuel_temp)
push!(lambda_RU,lambda_RU_temp)
push!(lambda_RD,lambda_RD_temp)
push!(lambda_sr,lambda_sr_temp)
push!(lambda_Pmax,lambda_Pmax_temp)
push!(lambda_Pmin,lambda_Pmin_temp)
push!(lambda_nsp,lambda_nsp_temp)
push!(lambda_regu,lambda_regu_temp)
push!(lambda_regd,lambda_regd_temp)

push!(lambda_prime_RU,lambda_prime_RU_temp)
push!(lambda_prime_RD,lambda_prime_RD_temp)
push!(lambda_prime_sr,lambda_prime_sr_temp)
push!(lambda_prime_Pmax,lambda_prime_Pmax_temp)
push!(lambda_prime_Pmin,lambda_prime_Pmin_temp)
push!(lambda_prime_nsp,lambda_prime_nsp_temp)
push!(lambda_prime_regu,lambda_prime_regu_temp)
push!(lambda_prime_regd,lambda_prime_regd_temp)

push!(lambda_ffr,lambda_ffr_temp)

push!(lambda_balance,lambda_balance_temp)
push!(lambda_transUB,lambda_transUB_temp)
push!(lambda_transLB,lambda_transLB_temp)
push!(lambda_RREGUreq,lambda_RREGUreq_temp)
push!(lambda_RREGDreq,lambda_RREGDreq_temp)
push!(lambda_NSPreq,lambda_NSPreq_temp)
push!(lambda_rfrrreq,lambda_rfrrreq_temp)
push!(lambda_frr,lambda_frr_temp)
push!(lambda_tffr1,lambda_tffr1_temp)
push!(lambda_tffr2,lambda_tffr2_temp)
push!(lambda_rfrr,lambda_rfrr_temp)
push!(lambda_inx1,lambda_inx1_temp)
push!(lambda_inx2,lambda_inx2_temp)
push!(lambda_inertia,lambda_inertia_temp)

dual_obj_budget_temp=@variable(relax[1],base_name = string("dual_obj_budget^",string(r)))
dual_obj_fuel_temp = @variable(relax[1],base_name = string("dual_obj_fuel^",string(r)))
dual_obj_capacity1_temp = @variable(relax[1],base_name = string("dual_obj_capacity1^",string(r)))
dual_obj_capacity1_prime_temp = @variable(relax[1],base_name = string("dual_obj_capacity1_prime^",string(r)))
dual_obj_capacity2_temp = @variable(relax[1],base_name = string("dual_obj_capacity2^",string(r)))
dual_obj_capacity2_prime_temp = @variable(relax[1],base_name = string("dual_obj_capcity2_prime^",string(r)))
dual_obj_ffr_temp = @variable(relax[1],base_name = string("dual_obj_ffr^",string(r)))
dual_obj_trans_temp = @variable(relax[1],base_name = string("dual_obj_trans^",string(r)))
dual_obj_req_temp = @variable(relax[1],base_name = string("dual_obj_req^",string(r)))
dual_obj_tffr_temp = @variable(relax[1],base_name = string("dual_obj_tffr^",string(r)))
dual_obj_rfrr_temp = @variable(relax[1],base_name = string("dual_obj_rfrr^",string(r)))
dual_obj_inertia1_temp = @variable(relax[1],base_name = string("dual_obj_inertia1^",string(r)))
dual_obj_inertia2_temp = @variable(relax[1],base_name = string("dual_obj_inertia2^",string(r)))
slack_temp = @variable(relax[1],base_name = string("slack^",string(r)))

push!(dual_obj_budget,dual_obj_budget_temp)
push!(dual_obj_fuel,dual_obj_fuel_temp )
push!(dual_obj_capacity1,dual_obj_capacity1_temp)
push!(dual_obj_capacity1_prime,dual_obj_capacity1_prime_temp)
push!(dual_obj_capacity2,dual_obj_capacity2_temp)
push!(dual_obj_capacity2_prime,dual_obj_capacity2_prime_temp)
push!(dual_obj_ffr,dual_obj_ffr_temp)
push!(dual_obj_trans,dual_obj_trans_temp)
push!(dual_obj_req ,dual_obj_req_temp )
push!(dual_obj_tffr,dual_obj_tffr_temp)
push!(dual_obj_rfrr,dual_obj_rfrr_temp )
push!(dual_obj_inertia1,dual_obj_inertia1_temp )
push!(dual_obj_inertia2,dual_obj_inertia2_temp)

push!(slack,slack_temp)

#CONSTRAINTS
#Primal Constraints

@constraint(relax[1],slack_temp==s_budget_minus_temp+sum(s_balance_plus_temp[t]+s_balance_minus_temp[t] for t=1:HORIZON)+sum(s_prime_Pmin_plus_temp[m,t] for m=1:QS,t=1:HORIZON)+sum(s_Pmin_plus_temp[n,t] for n=1:GENERATORS,t=1:HORIZON)+sum(s_transUB_minus_temp[l,t]+s_transLB_plus_temp[l,t] for l = 1:ARCS, t = 1:HORIZON)+sum(s_RREGU_plus_temp[t]+s_RREGD_plus_temp[t]+s_RNSP_plus_temp[t]+s_rfrr_plus_temp[t] for t = 1:HORIZON)+sum(s_tffr1_plus_temp[sg,t]+s_tffr2_minus_temp[sg,t]+s_inx1_plus_temp[sg,t]+s_inx2_minus_temp[sg,t] for sg=1:INTERVALS-1, t = 1:HORIZON))


#FUEL CONSTRAINTS


@constraint(relax[1],sum(c_temp[n,t] for n = 1:GENERATORS,t = 1:HORIZON)+sum(c_prime_temp[m,t] for m =1:QS,t = 1:HORIZON)-s_budget_minus_temp<= phi- (sum(SU[n]*x_hat[n,t]+SD[n]*z_hat[n,t] for t = 1:HORIZON, n=1:GENERATORS)+sum(SU_prime[m]*x_prime[m,t]+SD_prime[m]*z_prime[m,t] for t = 1:HORIZON, m=1:QS)))

#fuel
for t = 1:HORIZON
    for j = 1:BREAKPOINTS
        for n = 1:GENERATORS
            #fuel
           @constraint(relax[1],c_temp[n,t]-beta[n,j]*p_temp[n,t]>=alpha[n,j]*y_hat[n,t])   
        end
        for m = 1:QS
            #fuel'
            @constraint(relax[1],c_prime_temp[m,t]-beta_prime[m,j]*p_prime_temp[m,t]>=alpha_prime[m,j]*y_prime[m,t])
        end
    end
end



#CAPACITY CONSTRAINTS
for t=1:HORIZON
    #conventional
    for n = 1:GENERATORS
        if t>=2
        #RU
            @constraint(relax[1],p_temp[n,t]-p_temp[n,t-1] <= RU[n]*y_hat[n,t-1]+RU_bar[n]*x_hat[n,t])
            #RD
            @constraint(relax[1],p_temp[n,t-1]-p_temp[n,t]<=RD[n]*y_hat[n,t]+RD_bar[n]*z_hat[n,t])
        end
        #sr
        @constraint(relax[1],sr_temp[n,t]<=10*RMT[n]*y_hat[n,t])
        #Pmax
        @constraint(relax[1],p_temp[n,t]+sr_temp[n,t]+regu_temp[n,t]<=Pmax[n]*y_hat[n,t])
        #Pmin
        @constraint(relax[1],p_temp[n,t]-regd_temp[n,t]+s_Pmin_plus_temp[n,t]>=Pmin[n]*y_hat[n,t])
        #nsp
        @constraint(relax[1],nsp_temp[n,t]<=(1-y_hat[n,t])*NSP[n])
        #regu
        @constraint(relax[1],regu_temp[n,t]<=y_hat[n,t]*REGU[n])
        #regd        
        @constraint(relax[1],regd_temp[n,t]<=y_hat[n,t]*REGD[n])
    end
    #quick start
    for m= 1:QS
        if t>=2
            #RU'
            @constraint(relax[1],p_prime_temp[m,t]-p_prime_temp[m,t-1] <= RU_prime[m]*y_prime[m,t-1]+RU_bar_prime[m]*x_prime[m,t])
            #RD'
            @constraint(relax[1],p_prime_temp[m,t-1]-p_prime_temp[m,t]<=RD_prime[m]*y_prime[m,t]+RD_bar_prime[m]*z_prime[m,t])
        end
        #sr'
        @constraint(relax[1],sr_prime_temp[m,t]<=10*RMT_prime[m]*y_prime[m,t])
        #Pmax'
        @constraint(relax[1],p_prime_temp[m,t]+sr_prime_temp[m,t]+regu_prime_temp[m,t]<=Pmax_prime[m]*y_prime[m,t])
        #Pmin'
        @constraint(relax[1],p_prime_temp[m,t]-regd_prime_temp[m,t]+s_prime_Pmin_plus_temp[m,t]>=Pmin_prime[m]*y_prime[m,t])   
        #nsp'
        @constraint(relax[1],nsp_prime_temp[m,t]<=(1-y_prime[m,t])*NSP_prime[m])
        #regu
        @constraint(relax[1],regu_prime_temp[m,t]<=y_prime[m,t]*REGU_prime[m])
        #regd
        @constraint(relax[1],regd_prime_temp[m,t]<=y_prime[m,t]*REGD_prime[m])
    end

    for b=1:BUSES
       #ffr
        @constraint(relax[1],ffr_temp[b,t]<=FFR[b])
    end
end


#DEMAND BALANCE CONSTRAINTS
for t = 1:HORIZON
    #balance
    @constraint(relax[1],sum(p_temp[n,t] for n=1:GENERATORS)+  sum(p_prime_temp[m,t] for m=1:QS)+s_balance_plus_temp[t]-s_balance_minus_temp[t]==sum(demand_hat[b,t] for b=1:BUSES))
    #define p_bus for transmission constraints. P_bus = sum(p_n+p'_n) for n in Gb
    for b=1:BUSES
        if length(G[b])==0
            if length(G_prime[b])==0
                @constraint(relax[1],p_bus_temp[b,t]==0)
            else
                @constraint(relax[1],p_bus_temp[b,t]==sum(p_prime_temp[m,t] for m in G_prime[b]))
            end
        end

        if length(G[b])==0
            if length(G_prime[b])>0

            end
        end

        if length(G[b])>0
            if length(G_prime[b])==0
                @constraint(relax[1],p_bus_temp[b,t]==sum(p_temp[n,t] for n in G[b]))
            else
               @constraint(relax[1],p_bus_temp[b,t]==sum(p_temp[n,t] for n in G[b])+sum(p_prime_temp[m,t] for m in G_prime[b]))
            end
        end
    end

    #transmission constraints
    for l=1:ARCS
        #transUB
        @constraint(relax[1], sum(SF[b,l]*p_bus_temp[b,t] for b=1:BUSES) - s_transUB_minus_temp[l,t]<=sum(SF[b,l]*demand_hat[b,t] for b=1:BUSES)+F[l])
        #transLB
        @constraint(relax[1], sum(SF[b,l]*p_bus_temp[b,t] for b=1:BUSES) + s_transLB_plus_temp[l,t]>=sum(SF[b,l]*demand_hat[b,t] for b=1:BUSES)-F[l])
    end
end

#REQUIREMENT CONSTRAINTS
for t = 1:HORIZON
    #RREGUreq
    @constraint(relax[1],sum(regu_temp[n,t] for n = 1:GENERATORS)+sum(regu_prime_temp[m,t] for m=1:QS)+s_RREGU_plus_temp[t]>=RREGU[t])
    #RREGDreq
    @constraint(relax[1],sum(regd_temp[n,t] for n = 1:GENERATORS)+sum(regd_prime_temp[m,t] for m=1:QS)+s_RREGD_plus_temp[t]>=RREGD[t])
    #NSPreq
    @constraint(relax[1],sum(nsp_temp[n,t] for n = 1:GENERATORS)+sum(nsp_prime_temp[m,t] for m=1:QS)+s_RNSP_plus_temp[t]>=RNSP[t])
    #rfrrreq
    @constraint(relax[1],frr_temp[t]-rfrr_temp[t]+s_rfrr_plus_temp[t]>=0)
    #frr
    @constraint(relax[1],frr_temp[t]-tffr_temp[t]-sum(sr_temp[n,t] for n=1:GENERATORS)-sum(sr_prime_temp[m,t] for m=1:QS)==0)
end


#INERTIA CONSTRAINTS
for t=1:HORIZON
    for sg=1:INTERVALS-1
        #tffr1
        @constraint(relax[1],tffr_temp[t]-K[sg]*sum(ffr_temp[b,t] for b=1:BUSES)+s_tffr1_plus_temp[sg,t]>=-M_ffr*(1-delta[sg,t]+delta[sg+1,t]))
        #tffr2
        @constraint(relax[1],tffr_temp[t]-K[sg]*sum(ffr_temp[b,t] for b=1:BUSES)-s_tffr2_minus_temp[sg,t]<=M_ffr*(1-delta[sg,t]+delta[sg+1,t]))
        #inx1       
        @constraint(relax[1],inx_temp[sg,t]+s_inx1_plus_temp[sg,t]>=delta[sg+1,t]*(IN[sg+1]-IN[sg]))
        #inx2   
        @constraint(relax[1],inx_temp[sg,t]-s_inx2_minus_temp[sg,t]<=delta[sg,t]*(IN[sg+1]-IN[sg]))   
    end
    #rfrr
    @constraint(relax[1],rfrr_temp[t]-sum(IR[sg]*inx_temp[sg,t] for sg=1:INTERVALS-1)==RFRR_1) 
    #inertia 
    @constraint(relax[1],sum(inx_temp[sg,t] for sg = 1:INTERVALS-1) == sum(H[n]*S[n]*y_hat[n,t] for n=1:GENERATORS)+sum(H_prime[m]*S_prime[m]*y_prime[m,t] for m=1:QS)-IN[1]) 
end


#primal non-negativity
@constraint(relax[1],s_budget_minus_temp>=0)
for t = 1:HORIZON
    @constraint(relax[1],tffr_temp[t]>=0)
    @constraint(relax[1],frr_temp[t]>=0)
    @constraint(relax[1],rfrr_temp[t]>=0)
    @constraint(relax[1],s_balance_plus_temp[t]>=0)
    @constraint(relax[1],s_balance_minus_temp[t]>=0)
    @constraint(relax[1],s_RREGU_plus_temp[t]>=0)
    @constraint(relax[1],s_RREGD_plus_temp[t]>=0)
    @constraint(relax[1],s_RNSP_plus_temp[t]>=0)
    @constraint(relax[1],s_rfrr_plus_temp[t]>=0)
    for n = 1:GENERATORS
        @constraint(relax[1],c_temp[n,t]>=0)
        @constraint(relax[1],p_temp[n,t]>=0)
        @constraint(relax[1],regu_temp[n,t]>=0)
        @constraint(relax[1],regd_temp[n,t]>=0)
        @constraint(relax[1],sr_temp[n,t]>=0)
        @constraint(relax[1],nsp_temp[n,t]>=0)
        @constraint(relax[1],s_Pmin_plus_temp[n,t]>=0)
    end
    for m = 1:QS
        @constraint(relax[1],c_prime_temp[m,t]>=0)
        @constraint(relax[1],p_prime_temp[m,t]>=0)
        @constraint(relax[1],regu_prime_temp[m,t]>=0)
        @constraint(relax[1],regd_prime_temp[m,t]>=0)
        @constraint(relax[1],sr_prime_temp[m,t]>=0)
        @constraint(relax[1],nsp_prime_temp[m,t]>=0)
        @constraint(relax[1],s_prime_Pmin_plus_temp[m,t]>=0)
    end
    for b =1:BUSES
        @constraint(relax[1],ffr_temp[b,t]>=0)
    end
    for l = 1:ARCS
        @constraint(relax[1],s_transLB_plus_temp[l,t]>=0)
        @constraint(relax[1],s_transUB_minus_temp[l,t]>=0)
    end
    for sg=1:INTERVALS
        @constraint(relax[1],inx_temp[sg,t]>=0)
        if sg < INTERVALS
            @constraint(relax[1],s_tffr1_plus_temp[sg,t]>=0)
            @constraint(relax[1],s_tffr2_minus_temp[sg,t]>=0)
            @constraint(relax[1],s_inx1_plus_temp[sg,t]>=0)
            @constraint(relax[1],s_inx2_minus_temp[sg,t]>=0)
        end
    end
end


#DUAL CONSTRAINTS
for t=1:HORIZON

    for n =1:GENERATORS
        #c
        @constraint(relax[1],lambda_budget_temp+sum(lambda_fuel_temp[j,n,t] for j =1:BREAKPOINTS)<=0)
        #p
        if t==1    
            @constraint(relax[1],-sum(beta[n,j]*lambda_fuel_temp[j,n,2] for j=1:BREAKPOINTS)-lambda_RU_temp[n,2]+lambda_RD_temp[n,2]+lambda_Pmax_temp[n,1]+lambda_Pmin_temp[n,1]+lambda_balance_temp[1]+sum(SF[generator_data[1,n]]*(lambda_transUB_temp[l,1]+lambda_transLB_temp[l,1]) for l=1:ARCS)<=0)
        elseif t==T 
            @constraint(relax[1],-sum(beta[n,j]*lambda_fuel_temp[j,n,t] for j=1:BREAKPOINTS)+lambda_RU_temp[n,t]-lambda_RD_temp[n,t]+lambda_Pmax_temp[n,t]+lambda_Pmin_temp[n,t]+lambda_balance_temp[t]+sum(SF[generator_data[1,n]]*(lambda_transUB_temp[l,t]+lambda_transLB_temp[l,t]) for l=1:ARCS)<=0)    
        else
            @constraint(relax[1],-sum(beta[n,j]*lambda_fuel_temp[j,n,t] for j=1:BREAKPOINTS)+lambda_RU_temp[n,t]-lambda_RU_temp[n,t+1]-lambda_RD_temp[n,t]+lambda_RD_temp[n,t+1]+lambda_Pmax_temp[n,t]+lambda_Pmin_temp[n,t]+lambda_balance_temp[t]+sum(SF[generator_data[1,n]]*(lambda_transUB_temp[l,t]+lambda_transLB_temp[l,t]) for l=1:ARCS)<=0)
        end
            


        #regu
        @constraint(relax[1],lambda_Pmax_temp[n,t]+lambda_regu_temp[n,t]+lambda_RREGUreq_temp[t]<=0)       
        #regd
        @constraint(relax[1],-lambda_Pmin_temp[n,t]+lambda_regd_temp[n,t]+lambda_RREGDreq_temp[t]<=0)
        #sr
        @constraint(relax[1],lambda_sr_temp[n,t]+lambda_Pmax_temp[n,t]-lambda_frr_temp[t]<=0)
        #nsp
        @constraint(relax[1],lambda_nsp_temp[n,t]+lambda_NSPreq_temp[t]<=0)
    end
    for m = 1:QS
        #c'
        @constraint(relax[1],lambda_budget_temp+sum(lambda_prime_fuel_temp[j,m,t] for j =1:BREAKPOINTS)<=0)
        #p'
        if t==1
            @constraint(relax[1],-sum(beta_prime[m,j]*lambda_prime_fuel_temp[j,m,t] for j=1:BREAKPOINTS)-lambda_prime_RU_temp[m,t+1]+lambda_prime_RD_temp[m,t+1]+lambda_prime_Pmax_temp[m,t]+lambda_prime_Pmin_temp[m,t]+lambda_balance_temp[t]+sum(SF[generator_data[1,m]]*(lambda_transUB_temp[l,t]+lambda_transLB_temp[l,t]) for l=1:ARCS)<=0)
        elseif t==T
            @constraint(relax[1],-sum(beta_prime[m,j]*lambda_prime_fuel_temp[j,m,t] for j=1:BREAKPOINTS)+lambda_prime_RU_temp[m,t]-lambda_prime_RD_temp[m,t]+lambda_prime_Pmax_temp[m,t]+lambda_prime_Pmin_temp[m,t]+lambda_balance_temp[t]+sum(SF[generator_data[1,m]]*(lambda_transUB_temp[l,t]+lambda_transLB_temp[l,t]) for l=1:ARCS)<=0)
        else
            @constraint(relax[1],-sum(beta_prime[m,j]*lambda_prime_fuel_temp[j,m,t] for j=1:BREAKPOINTS)+lambda_prime_RU_temp[m,t]-lambda_prime_RU_temp[m,t+1]-lambda_prime_RD_temp[m,t]+lambda_prime_RD_temp[m,t+1]+lambda_prime_Pmax_temp[m,t]+lambda_prime_Pmin_temp[m,t]+lambda_balance_temp[t]+sum(SF[generator_data[1,m]]*(lambda_transUB_temp[l,t]+lambda_transLB_temp[l,t]) for l=1:ARCS)<=0)
        end
            
        #regu'
        @constraint(relax[1],lambda_prime_Pmax_temp[m,t]+lambda_prime_regu_temp[m,t]+lambda_RREGUreq_temp[t]<=0)       
        #regd'
        @constraint(relax[1],-lambda_prime_Pmin_temp[m,t]+lambda_prime_regd_temp[m,t]+lambda_RREGDreq_temp[t]<=0)
        #sr'
        @constraint(relax[1],lambda_prime_sr_temp[m,t]+lambda_prime_Pmax_temp[m,t]-lambda_frr_temp[t]<=0)
        #nsp'
        @constraint(relax[1],lambda_prime_nsp_temp[m,t]+lambda_NSPreq_temp[t]<=0)
    end

    for b=1:BUSES
        #ffr
        @constraint(relax[1],lambda_ffr_temp[b,t]-sum(K[sg]*(lambda_tffr1_temp[sg,t]+lambda_tffr2_temp[sg,t]) for sg=1:INTERVALS-1)<=0)
    end
    #tffr
    @constraint(relax[1],-lambda_frr_temp[t]+sum(lambda_tffr1_temp[sg,t]+lambda_tffr2_temp[sg,t] for sg=1:INTERVALS-1)<=0)
    #frr
    @constraint(relax[1],lambda_rfrrreq_temp[t]+lambda_frr_temp[t]<=0)
    #rfrr
    @constraint(relax[1],-lambda_rfrrreq_temp[t]+lambda_rfrr_temp[t]<=0)
    for sg = 1:INTERVALS-1
        #inx
        @constraint(relax[1],lambda_inx1_temp[sg,t]+lambda_inx2_temp[sg,t]-IR[sg]*lambda_rfrr_temp[t]+lambda_inertia_temp[t]<=0)
    end

end




#SLACK DUAL CONSTRAINTS
#budget 
@constraint(relax[1],lambda_budget_temp>=-1)

for t=1:HORIZON
    #balance 
    @constraint(relax[1],-1<=lambda_balance_temp[t]<=1)
    #RREGUreq
    @constraint(relax[1],lambda_RREGUreq_temp[t]<=1)
    #RREGDreq
    @constraint(relax[1],lambda_RREGDreq_temp[t]<=1)
    #NSPreq
    @constraint(relax[1],lambda_NSPreq_temp[t]<=1)
    #rfrrreq
    @constraint(relax[1],lambda_rfrrreq_temp[t]<=1)
    for n=1:GENERATORS
        @constraint(relax[1],lambda_Pmin_temp[n,t]<=1)
    end
    for m=1:QS
        @constraint(relax[1],lambda_prime_Pmin_temp[m,t]<=1)
    end

    for l =1:ARCS
        #transUB, transLB 
        @constraint(relax[1],lambda_transUB_temp[l,t]>=-1)
        @constraint(relax[1],lambda_transLB_temp[l,t]<=1) 
    end
    for sg=1:INTERVALS-1
        #tffr1,tffr2 
        @constraint(relax[1],lambda_tffr1_temp[sg,t]<=1)
        @constraint(relax[1],lambda_tffr2_temp[sg,t]>=-1)
        #inx1,inx2 
        @constraint(relax[1],lambda_inx1_temp[sg,t]<=1)
        @constraint(relax[1],lambda_inx2_temp[sg,t]>=-1)
    end
end

@constraint(relax[1],lambda_budget_temp <=0)
for t =1:HORIZON
    @constraint(relax[1],lambda_RREGUreq_temp[t] >=0)
    @constraint(relax[1],lambda_RREGDreq_temp[t] >=0)
    @constraint(relax[1],lambda_NSPreq_temp[t]>=0)
    @constraint(relax[1],lambda_rfrrreq_temp[t]>=0)
    #lambda_balance_temp unrestricted
    #lambda_frr_temp unrestricted
    #lambda_rfrr_temp unrestricted
    #lambda_inertia_temp unrestricted
    for n=1:GENERATORS
        @constraint(relax[1],lambda_RU_temp[n,t]<=0)
        @constraint(relax[1],lambda_RD_temp[n,t]<=0)
        @constraint(relax[1],lambda_sr_temp[n,t]<=0)

        @constraint(relax[1],lambda_Pmax_temp[n,t]<=0)
        @constraint(relax[1],lambda_Pmin_temp[n,t]>=0)
        @constraint(relax[1],lambda_nsp_temp[n,t]<=0)
        @constraint(relax[1],lambda_regu_temp[n,t]<=0)
        @constraint(relax[1],lambda_regd_temp[n,t]<=0)
        for j = 1:BREAKPOINTS
            @constraint(relax[1],lambda_fuel_temp[j,n,t]>=0)
        end
    end

    for m=1:QS
        @constraint(relax[1],lambda_prime_RU_temp[m,t]<=0)
        @constraint(relax[1],lambda_prime_RD_temp[m,t]<=0)
        @constraint(relax[1],lambda_prime_sr_temp[m,t]<=0)
        @constraint(relax[1],lambda_prime_Pmax_temp[m,t]<=0)
        @constraint(relax[1],lambda_prime_Pmin_temp[m,t]>=0)
        @constraint(relax[1],lambda_prime_nsp_temp[m,t]<=0)
        @constraint(relax[1],lambda_prime_regu_temp[m,t]<=0)
        @constraint(relax[1],lambda_prime_regd_temp[m,t]<=0)
        for j=1:BREAKPOINTS
            @constraint(relax[1],lambda_prime_fuel_temp[j,m,t]>=0)
        end
    end

    for b = 1:BUSES
        @constraint(relax[1],lambda_ffr_temp[b,t]<=0)
    end

    for l = 1:ARCS
        @constraint(relax[1],lambda_transUB_temp[l,t]<=0)
        @constraint(relax[1],lambda_transLB_temp[l,t]>=0)
    end
    for sg=1:INTERVALS-1
        @constraint(relax[1],lambda_tffr1_temp[sg,t]>=0)
        @constraint(relax[1],lambda_inx1_temp[sg,t]>=0)
        @constraint(relax[1],lambda_tffr2_temp[sg,t]<=0)
        @constraint(relax[1],lambda_inx2_temp[sg,t]<=0)
    end
end


#strong duality constraint
@constraint(relax[1],dual_obj_budget_temp+dual_obj_fuel_temp+dual_obj_capacity1_temp+dual_obj_capacity1_prime_temp+dual_obj_capacity2_temp+dual_obj_capacity2_prime_temp+dual_obj_ffr_temp+dual_obj_trans_temp+dual_obj_req_temp+dual_obj_tffr_temp+dual_obj_rfrr_temp+dual_obj_inertia1_temp+dual_obj_inertia2_temp ==slack_temp)
@constraint(relax[1],dual_obj_budget_temp==(phi-(sum(sum(SU[n]*x_hat[n,t] +SD[n]*z_hat[n,t] for n=1:GENERATORS)+sum(SU_prime[m]*x_prime[m,t]+SD_prime[m]*z_prime[m,t] for m=1:QS) for t=1:HORIZON)))*lambda_budget_temp)
@constraint(relax[1],dual_obj_fuel_temp==sum(sum(alpha[n,j]*y_hat[n,t]*lambda_fuel_temp[j,n,t] for n=1:GENERATORS)+sum(alpha_prime[m,j]*y_prime[m,t]*lambda_prime_fuel_temp[j,m,t] for m=1:QS) for j=1:BREAKPOINTS, t=1:HORIZON))
@constraint(relax[1],dual_obj_capacity1_temp==sum((RU[n]*y_hat[n,t-1]+RU_bar[n]*x_hat[n,t])*lambda_RU_temp[n,t]+(RD[n]*y_hat[n,t]+RD_bar[n]*z_hat[n,t])*lambda_RD_temp[n,t] for n=1:GENERATORS,t=2:HORIZON))
@constraint(relax[1],dual_obj_capacity2_temp==sum(y_hat[n,t]*(10*RMT[n]*lambda_sr_temp[n,t]+Pmax[n]*lambda_Pmax_temp[n,t]+Pmin[n]*lambda_Pmin_temp[n,t]+REGU[n]*lambda_regu_temp[n,t]+REGD[n]*lambda_regd_temp[n,t])+(1-y_hat[n,t])*NSP[n]*lambda_nsp_temp[n,t] for t=1:HORIZON,n=1:GENERATORS))
@constraint(relax[1],dual_obj_capacity1_prime_temp==sum((RU_prime[m]*y_prime[m,t-1]+RU_bar_prime[m]*x_prime[m,t])*lambda_prime_RU_temp[m,t]+(RD_prime[m]*y_prime[m,t]+RD_bar_prime[m]*z_prime[m,t])*lambda_prime_RD_temp[m,t] for m=1:QS,t=2:HORIZON))
@constraint(relax[1],dual_obj_capacity2_prime_temp==sum(y_prime[m,t]*(10*RMT_prime[m]*lambda_prime_sr_temp[m,t]+Pmax_prime[m]*lambda_prime_Pmax_temp[m,t]+Pmin[m]*lambda_prime_Pmin_temp[m,t]+REGU[m]*lambda_prime_regu_temp[m,t]+REGD_prime[m]*lambda_prime_regd_temp[m,t])+(1-y_prime[m,t])*NSP_prime[m]*lambda_prime_nsp_temp[m,t] for t=1:HORIZON,m=1:QS))
@constraint(relax[1],dual_obj_ffr_temp==sum(FFR[b]*lambda_ffr_temp[b,t] for b=1:BUSES, t=1:HORIZON))
@constraint(relax[1],dual_obj_trans_temp==sum( sum(demand_hat[b,t]*lambda_balance_temp[t] for b=1:BUSES) for t=1:HORIZON)+sum(sum(SF[b,l]*demand_hat[b,t]+F[l] for b=1:BUSES)*lambda_transUB_temp[l,t] +sum(SF[b,l]*demand_hat[b,t]-F[l] for b=1:BUSES)*lambda_transLB_temp[l,t] for t=1:HORIZON,l=1:ARCS))
@constraint(relax[1],dual_obj_req_temp==sum(RREGU[t]*lambda_RREGUreq_temp[t]+RREGD[t]*lambda_RREGDreq_temp[t] +RNSP[t]*lambda_NSPreq_temp[t] for t=1:HORIZON))
@constraint(relax[1],dual_obj_tffr_temp==sum(M_ffr*(1-delta[sg,t]+delta[sg+1,t])*(-lambda_tffr1_temp[sg,t]+lambda_tffr2_temp[sg,t]) for sg=1:INTERVALS-1, t=1:HORIZON))
@constraint(relax[1],dual_obj_rfrr_temp==sum(RFRR_1*lambda_rfrr_temp[t] for t=1:HORIZON))
@constraint(relax[1],dual_obj_inertia1_temp==sum(delta[sg+1,t]*(IN[sg+1]-IN[sg])*lambda_inx1_temp[sg,t]+delta[sg,t]*(IN[sg+1]-IN[sg])*lambda_inx2_temp[sg,t] for t=1:HORIZON,sg=1:INTERVALS-1))
@constraint(relax[1],dual_obj_inertia2_temp == sum((sum(H[n]*S[n]*y_hat[n,t] for n=1:GENERATORS)+sum(H_prime[m]*S_prime[m]*y_prime[m,t] for m=1:QS)-IN[1])*lambda_inertia_temp[t] for t =1:HORIZON))




println("======================================RELAXATION PROBLEM==========================================")

#println(relax[1])<----this code prints out the entire formulation


@constraint(relax[1],eta<=slack_temp)
@objective(relax[1],Max,eta)


println("------------------------------- relxation problem solution for iteration",r,":-------------------------------")

#store demand value
#demand_hat = JuMP.value.(demand)

println("demand: ",demand_hat)
for t= 1:HORIZON
    println("demand[",t,"]=",sum(demand_hat[:,t]))
end
println("total demand: ", sum(demand_hat))

#results
for i =1:r
    println("slack ",i,"out of",r," ",JuMP.value(slack[i]))
    println(JuMP.value(slack_temp))
end
println("objective value, eta=",JuMP.value.(eta))

println("------------------------------------------------------------------------------------------------------------")
push!(relax_obj,objective_value(relax[1]))     


r=r+1 #update iteration counter


HELLO DARKNESS
MY OLD FRIEND
======================================RELAXATION PROBLEM==========================================
------------------------------- relxation problem solution for iteration1:-------------------------------
demand: [300.0 350.0 200.0; 225.0 250.0 200.0; 200.0 250.0 225.0; 175.0 200.0 175.0; 150.0 200.0 175.0000000000004; 125.0 200.0 150.0]
demand[1]=1175.0
demand[2]=1450.0
demand[3]=1125.0000000000005
total demand: 3750.0000000000005
slack 1out of1 0.0
0.0
objective value, eta=0.0
------------------------------------------------------------------------------------------------------------


2

In [8]:
println("demand:",demand_hat)
println("x'",x_prime)
println("y'",y_prime)
println("z'",z_prime)
println("delta",delta)

demand:[300.0 350.0 200.0; 225.0 250.0 200.0; 200.0 250.0 225.0; 175.0 200.0 175.0; 150.0 200.0 175.0000000000004; 125.0 200.0 150.0]
x'[1.0 0.0 0.0; 0.0 0.0 0.0; 1.0 0.0 0.0; 1.0 0.0 0.0; 1.0 0.0 0.0; 1.0 0.0 0.0]
y'[1.0 1.0 1.0; 0.0 0.0 0.0; 1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0]
z'[0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
delta[1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0; 1.0 1.0 1.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]


In [9]:
#debugging for primal

println("PRIMAL RESULTS")
println("slack variables:")
println("total slack: ",JuMP.value.(slack_temp))
println("budget: ",sum(JuMP.value.(s_budget_minus_temp))) #slack variable
println("balance+: ",sum(JuMP.value.(s_balance_plus_temp)))
println("balance-: ",sum(JuMP.value.(s_balance_minus_temp)))
println("Pmin: ",sum(JuMP.value.(s_Pmin_plus_temp)))
println("Pmin': ",sum(JuMP.value.(s_prime_Pmin_plus_temp)))
println("transUB: ",sum(JuMP.value.(s_transUB_minus_temp)))
println("transLB: ",sum(JuMP.value.(s_transLB_plus_temp)))
println("RREGU: ",sum(JuMP.value.(s_RREGU_plus_temp)))
println("RREGD: ",sum(JuMP.value.(s_RREGD_plus_temp)))
println("RNSP: ",sum(JuMP.value.(s_RNSP_plus_temp)))
println("rfrr: ",sum(JuMP.value.(s_rfrr_plus_temp)))
println("tffr1: ",sum(JuMP.value.(s_tffr1_plus_temp)))
println("tffr2: ",sum(JuMP.value.(s_tffr2_minus_temp)))
println("inx1: ",sum(JuMP.value.(s_inx1_plus_temp)))
println("inx2: ",sum(JuMP.value.(s_inx2_minus_temp)))
println("s_balance+: ", JuMP.value.(s_balance_plus_temp))
println("s_balance-: ", JuMP.value.(s_balance_minus_temp))
println("other primal variables:")
println("p': ", JuMP.value.(p_prime_temp))
println("inx: ", JuMP.value.(inx_temp))
println("demand: ", demand_hat)


PRIMAL RESULTS
slack variables:
total slack: 0.0
budget: 0.0
balance+: 0.0
balance-: 0.0
Pmin: 0.0
Pmin': 0.0
transUB: 0.0
transLB: 0.0
RREGU: 0.0
RREGD: 0.0
RNSP: 0.0
rfrr: 0.0
tffr1: 0.0
tffr2: 0.0
inx1: 0.0
inx2: 0.0
s_balance+: [0.0, 0.0, 0.0]
s_balance-: [0.0, 0.0, 0.0]
other primal variables:
p': [193.74981440873972 324.99999441206455 100.0; 0.0 0.0 0.0; 200.0 250.00000000000006 426.52602917708543; 131.25018284386547 100.0 100.00000000000006; 243.0618077897238 293.1972170416115 316.6711822765911; 406.938194957671 481.80278854632394 181.80278854632394]
inx: [120.0 120.0 120.0; 16.0 16.0 16.0; 16.0 16.0 16.0; 25.0 25.0 25.0; 25.0 25.0 25.0; 28.0 28.0 28.0; 20.0 20.0 20.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
demand: [300.0 350.0 200.0; 225.0 250.0 200.0; 200.0 250.0 225.0; 175.0 200.0 175.0; 150.0 200.0 175.0000000000004; 125.0 200.0 150.0]


In [10]:
#debugging for dual
#dual variables

println("------------------------dual variables------------------------")
println("budget: ",JuMP.value.(lambda_budget_temp))
println("fuel: ",JuMP.value.(lambda_fuel_temp))
println("fuel': ",JuMP.value.(lambda_prime_fuel_temp))
println("RU: ",JuMP.value.(lambda_RU_temp))
println("RU': ",JuMP.value.(lambda_prime_RU_temp))
println("RD : ",JuMP.value.(lambda_RD_temp))
println("RD': ",JuMP.value.(lambda_prime_RD_temp))
println("sr: ",JuMP.value.(lambda_sr_temp))
println("sr': ",JuMP.value.(lambda_prime_sr_temp))
println("Pmax: ",JuMP.value.(lambda_Pmax_temp))
println("Pmax': ",JuMP.value.(lambda_prime_Pmax_temp))
println("Pmin: ",JuMP.value.(lambda_Pmin_temp))
println("Pmin': ",JuMP.value.(lambda_prime_Pmin_temp))
println("nsp: ",JuMP.value.(lambda_nsp_temp))
println("nsp': ",JuMP.value.(lambda_prime_nsp_temp))
println("regu: ", JuMP.value.(lambda_regu_temp))
println("regu': ",JuMP.value.(lambda_prime_regu_temp))
println("regd: ",JuMP.value.(lambda_regd_temp))
println("regd': ",JuMP.value.(lambda_prime_regd_temp))
println("ffr: ",JuMP.value.(lambda_ffr_temp))
println("balance: ",JuMP.value.(lambda_balance_temp))
println("transUB: ",JuMP.value.(lambda_transUB_temp))
println("transLB: ",JuMP.value.(lambda_transLB_temp))
println("RREGUreq: ",JuMP.value.(lambda_RREGUreq_temp))
println("RREGDreq: ",JuMP.value.(lambda_RREGDreq_temp))
println("NSPreq: ",JuMP.value.(lambda_NSPreq_temp))
println("rfrr-req: ",JuMP.value.(lambda_rfrrreq_temp))
println("frr: ",JuMP.value.(lambda_frr_temp))
println("tffr1: ",JuMP.value.(lambda_tffr1_temp))
println("tffr2: ",JuMP.value.(lambda_tffr2_temp))
println("rfrr: ",JuMP.value.(lambda_rfrr_temp))
println("inx1: ",JuMP.value.(lambda_inx1_temp))
println("inx2: ",JuMP.value.(lambda_inx2_temp))
println("inertia: ",JuMP.value.(lambda_inertia_temp))
println("------------------------------------")
println("(y': ",y_prime,")")
println("gamma+: ", gamma_plus_hat)
println("gamma-: ", gamma_minus_hat)
println("d_hat: ", d_hat)
println("F: ",F)

println("F",F)

------------------------dual variables------------------------
budget: 0.0
fuel: [0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0]

[0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0]

[0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0]
fuel': [0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0]

[0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0]

[0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0]
RU: [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0]
RU': [0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0; 0.0 0.0 0.0

In [11]:
#